# Traveling Ethiopia Search Problem Question #3

This solution implements the A (A-Star) Search algorithm* to find the optimal path from Addis Ababa to Moyale.
Data Structure AnalysisTo perform A* Search, we need two distinct sets of data derived from Figure 3:Backward Cost ($g(n)$): The actual travel cost between cities (represented by the numbers on the edges).

This is stored in an adjacency dictionary.Heuristic Value ($h(n)$): The estimated distance from a specific city to the goal state (Moyale). This is represented by the numbers inside the city nodes.The A* algorithm uses the formula $f(n) = g(n) + h(n)$ to prioritize which city to explore next, ensuring the search is both optimal and efficient.

In [ ]:
import heapq

# 1. GRAPH REPRESENTATION (Backward Costs g(n))
# Based on the edge weights in Figure 3.
ethiopia_map_weighted = {
    'Addis Ababa': {'Adama': 3, 'Ambo': 5, 'Debre Birhan': 5},
    'Adama': {'Addis Ababa': 3, 'Matahara': 3, 'Asella': 4, 'Batu': 4},
    'Ambo': {'Addis Ababa': 5, 'Wolkite': 6, 'Nekemte': 9},
    'Debre Birhan': {'Addis Ababa': 5, 'Debre Sina': 2},
    'Debre Sina': {'Debre Birhan': 2, 'Kemise': 7, 'Debre Markos': 17},
    'Debre Markos': {'Debre Sina': 17, 'Finote Selam': 3},
    'Kemise': {'Debre Sina': 7, 'Dessie': 4},
    'Dessie': {'Kemise': 4, 'Woldia': 6},
    'Woldia': {'Dessie': 6, 'Lalibela': 7, 'Alamata': 3, 'Samara': 8},
    'Lalibela': {'Woldia': 7, 'Debre Tabor': 8, 'Sekota': 6},
    'Sekota': {'Lalibela': 6, 'Mekelle': 9, 'Alamata': 6},
    'Debre Tabor': {'Lalibela': 8, 'Bahir Dar': 4},
    'Bahir Dar': {'Debre Tabor': 4, 'Finote Selam': 6, 'Injibara': 4, 'Metekel': 11, 'Azezo': 7},
    'Finote Selam': {'Bahir Dar': 6, 'Debre Markos': 3, 'Injibara': 2},
    'Injibara': {'Bahir Dar': 4, 'Finote Selam': 2},
    'Metekel': {'Bahir Dar': 11},
    'Azezo': {'Bahir Dar': 7, 'Gondar': 1, 'Metema': 7},
    'Gondar': {'Azezo': 1, 'Humera': 9, 'Metema': 7, 'Debarke': 4},
    'Metema': {'Azezo': 7, 'Gondar': 7, 'Kartum': 19},
    'Kartum': {'Metema': 19, 'Humera': 21},
    'Humera': {'Kartum': 21, 'Gondar': 9, 'Shire': 8},
    'Shire': {'Humera': 8, 'Debarke': 2, 'Axum': 2},
    'Debarke': {'Gondar': 4, 'Shire': 2},
    'Axum': {'Shire': 2, 'Adwa': 1, 'Asmara': 5},
    'Adwa': {'Axum': 1, 'Adigrat': 4, 'Mekelle': 7},
    'Asmara': {'Axum': 5, 'Adigrat': 6},
    'Adigrat': {'Asmara': 6, 'Adwa': 4, 'Mekelle': 7},
    'Mekelle': {'Adwa': 7, 'Adigrat': 7, 'Sekota': 9, 'Alamata': 5},
    'Alamata': {'Mekelle': 5, 'Sekota': 6, 'Woldia': 3, 'Samara': 11},
    'Samara': {'Woldia': 8, 'Alamata': 11, 'Fanti Rasu': 7, 'Gabi Rasu': 9},
    'Fanti Rasu': {'Samara': 7, 'Kilbet Rasu': 6},
    'Kilbet Rasu': {'Fanti Rasu': 6},
    'Gabi Rasu': {'Samara': 9, 'Awash': 5},
    'Awash': {'Gabi Rasu': 5, 'Matahara': 1, 'Chiro': 4},
    'Matahara': {'Adama': 3, 'Awash': 1},
    'Chiro': {'Awash': 4, 'Dire Dawa': 8},
    'Dire Dawa': {'Chiro': 8, 'Harar': 4},
    'Harar': {'Dire Dawa': 4, 'Babile': 2},
    'Babile': {'Harar': 2, 'Jigjiga': 3},
    'Jigjiga': {'Babile': 3, 'Dega Habur': 5},
    'Dega Habur': {'Jigjiga': 5, 'Kebri Dehar': 6},
    'Kebri Dehar': {'Dega Habur': 6, 'Gode': 5, 'Werder': 6},
    'Werder': {'Kebri Dehar': 6},
    'Gode': {'Kebri Dehar': 5, 'Dollo': 17, 'Mokadisho': 22, 'Sof Oumer': 23},
    'Dollo': {'Gode': 17},
    'Mokadisho': {'Gode': 22},
    'Batu': {'Adama': 4, 'Buta Jirra': 2, 'Shashemene': 3},
    'Buta Jirra': {'Batu': 2, 'Worabe': 2},
    'Worabe': {'Buta Jirra': 2, 'Wolkite': 5, 'Hossana': 2},
    'Wolkite': {'Ambo': 6, 'Worabe': 5, 'Jimma': 8},
    'Jimma': {'Wolkite': 8, 'Bedelle': 9, 'Bonga': 4},
    'Bedelle': {'Jimma': 9, 'Nekemte': 5, 'Gore': 6},
    'Nekemte': {'Ambo': 9, 'Bedelle': 5, 'Gimbi': 4},
    'Gimbi': {'Nekemte': 4, 'Dembi Dollo': 6},
    'Dembi Dollo': {'Gimbi': 6, 'Assosa': 12, 'Gambella': 4},
    'Assosa': {'Dembi Dollo': 12, 'Metekel': 8},
    'Gambella': {'Dembi Dollo': 4, 'Gore': 5},
    'Gore': {'Gambella': 5, 'Bedelle': 6, 'Tepi': 9},
    'Tepi': {'Gore': 9, 'Bonga': 8, 'Mezan Teferi': 4},
    'Bonga': {'Jimma': 4, 'Tepi': 8, 'Dawro': 10, 'Mezan Teferi': 4},
    'Mezan Teferi': {'Tepi': 4, 'Bonga': 4},
    'Dawro': {'Bonga': 10, 'Wolaita Sodo': 6},
    'Wolaita Sodo': {'Dawro': 6, 'Hossana': 4, 'Arba Minch': 5},
    'Hossana': {'Worabe': 2, 'Wolaita Sodo': 4, 'Shashemene': 7},
    'Shashemene': {'Batu': 3, 'Hossana': 7, 'Hawassa': 1, 'Dodola': 3},
    'Hawassa': {'Shashemene': 1, 'Dilla': 3},
    'Dilla': {'Hawassa': 3, 'Bule Hora': 4},
    'Bule Hora': {'Dilla': 4, 'Yabello': 3},
    'Yabello': {'Bule Hora': 3, 'Konso': 3, 'Moyale': 6},
    'Moyale': {'Yabello': 6, 'Nairobi': 22},
    'Nairobi': {'Moyale': 22},
    'Konso': {'Yabello': 3, 'Arba Minch': 4},
    'Arba Minch': {'Wolaita Sodo': 5, 'Konso': 4, 'Basketo': 10},
    'Basketo': {'Arba Minch': 10, 'Bench Maji': 5},
    'Bench Maji': {'Basketo': 5, 'Juba': 22},
    'Juba': {'Bench Maji': 22},
    'Asella': {'Adama': 4, 'Assasa': 4},
    'Assasa': {'Asella': 4, 'Dodola': 1},
    'Dodola': {'Assasa': 1, 'Shashemene': 3, 'Bale': 13},
    'Bale': {'Dodola': 13, 'Goba': 18, 'Sof Oumer': 23, 'Liben': 11},
    'Goba': {'Bale': 18, 'Sof Oumer': 6},
    'Sof Oumer': {'Bale': 23, 'Goba': 6, 'Gode': 23},
    'Liben': {'Bale': 11}
}

# 2. HEURISTIC VALUES h(n)
# Based on the numbers inside the nodes in Figure 3.
ethiopia_heuristics = {
    'Addis Ababa': 26, 'Adama': 23, 'Ambo': 31, 'Debre Birhan': 31, 'Debre Sina': 33,
    'Debre Markos': 39, 'Kemise': 40, 'Dessie': 44, 'Woldia': 50, 'Lalibela': 57,
    'Sekota': 59, 'Debre Tabor': 52, 'Bahir Dar': 48, 'Finote Selam': 42,
    'Injibara': 44, 'Metekel': 59, 'Azezo': 55, 'Gondar': 56, 'Metema': 62,
    'Kartum': 81, 'Humera': 65, 'Shire': 67, 'Debarke': 60, 'Axum': 66,
    'Adwa': 65, 'Asmara': 68, 'Adigrat': 62, 'Mekelle': 58, 'Alamata': 53,
    'Samara': 42, 'Fanti Rasu': 49, 'Kilbet Rasu': 55, 'Gabi Rasu': 32,
    'Awash': 27, 'Chiro': 31, 'Matahara': 26, 'Dire Dawa': 31, 'Harar': 35,
    'Babile': 37, 'Jigjiga': 40, 'Dega Habur': 45, 'Kebri Dehar': 40,
    'Werder': 46, 'Gode': 35, 'Dollo': 18, 'Mokadisho': 40, 'Sof Oumer': 45,
    'Bale': 22, 'Goba': 40, 'Dodola': 19, 'Assasa': 18, 'Asella': 22,
    'Batu': 19, 'Buta Jirra': 21, 'Worabe': 22, 'Wolkite': 25, 'Jimma': 33,
    'Bedelle': 40, 'Nekemte': 39, 'Gimbi': 43, 'Dembi Dollo': 49,
    'Assosa': 51, 'Gambella': 51, 'Gore': 46, 'Tepi': 41, 'Bonga': 33,
    'Mezan Teferi': 37, 'Dawro': 23, 'Wolaita Sodo': 17, 'Hossana': 21,
    'Shashemene': 16, 'Hawassa': 15, 'Dilla': 12, 'Bule Hora': 8,
    'Yabello': 6, 'Moyale': 0, 'Nairobi': 22, 'Konso': 9, 'Arba Minch': 13,
    'Basketo': 23, 'Bench Maji': 28, 'Juba': 50, 'Liben': 11
}

# 3. A* SEARCH ALGORITHM CLASS
class AStarSearch:
    def __init__(self, graph, heuristics):
        self.graph = graph
        self.heuristics = heuristics

    def search(self, start, goal):
        """
        Executes A* Search from start to goal.

        Args:
            start (str): The starting city.
            goal (str): The destination city.

        Returns:
            tuple: (path_list, total_cost) or (None, infinity) if failed.
        """
        # Priority queue stores tuples: (f_score, current_node)
        # f_score = g_score + h_score
        open_set = []
        heapq.heappush(open_set, (0 + self.heuristics.get(start, 0), start))

        # To reconstruct the path
        came_from = {}

        # Cost from start to node n
        g_score = {node: float('inf') for node in self.graph}
        g_score[start] = 0

        # Estimated total cost from start to goal through n
        f_score = {node: float('inf') for node in self.graph}
        f_score[start] = self.heuristics.get(start, 0)

        visited = set()

        while open_set:
            # Get node with lowest f_score
            current_f, current = heapq.heappop(open_set)

            if current == goal:
                return self._reconstruct_path(came_from, current), g_score[goal]

            # Skip if we found a better path to this node already
            # (Standard optimization for lazy dijkstra/A*)
            if current in visited and g_score[current] > current_f:
                 continue
            visited.add(current)

            neighbors = self.graph.get(current, {})
            for neighbor, weight in neighbors.items():
                tentative_g = g_score[current] + weight

                if tentative_g < g_score[neighbor]:
                    # This path to neighbor is better than any previous one
                    came_from[neighbor] = current
                    g_score[neighbor] = tentative_g
                    f_score[neighbor] = tentative_g + self.heuristics.get(neighbor, 0)

                    # Add to priority queue
                    heapq.heappush(open_set, (f_score[neighbor], neighbor))

        return None, float('inf')

    def _reconstruct_path(self, came_from, current):
        path = [current]
        while current in came_from:
            current = came_from[current]
            path.append(current)
        path.reverse()
        return path

# 4. EXECUTION
# Initialize the solver with the weighted map and heuristics
solver = AStarSearch(ethiopia_map_weighted, ethiopia_heuristics)

# Run A* Search
start_city = 'Addis Ababa'
goal_city = 'Moyale'

print(f"--- A* Search from {start_city} to {goal_city} ---")
path, cost = solver.search(start_city, goal_city)

if path:
    print(f"Optimal Path found: {path}")
    print(f"Total Cost: {cost}")
else:
    print("No path found.")

--- A* Search from Addis Ababa to Moyale ---
Optimal Path found: ['Addis Ababa', 'Adama', 'Batu', 'Shashemene', 'Hawassa', 'Dilla', 'Bule Hora', 'Yabello', 'Moyale']
Total Cost: 27
